In [8]:
import sys
import argparse
from Bio import Entrez
from Bio import SeqIO
import pandas as pd

In [7]:
family_file='/fs/project/PAS1117/ricardo/ssDNA_test/test_data/accession_not_found_refseq.csv'
dir_to_download='/fs/scratch/Sullivan_Lab/Ricardo/ssDNA_db/'

In [9]:
df_family = pd.read_csv(family_file)

In [15]:
Entrez.email = 'pavan.4@osu.edu'

In [ ]:
df_family

In [ ]:
# Create output directory if it doesn't exist
os.makedirs(dir_to_download, exist_ok=True)

In [13]:
for _, row in df_family.iterrows():
    project, family = row.iloc[0], row.iloc[1]  # Read Accession and Family

    print(f"Processing: Accession = {project}, Family = {family}")

Processing: Accession = KT149412, Family = Adamaviridae
Processing: Accession = KJ547648, Family = Adamaviridae
Processing: Accession = KP153485, Family = Adamaviridae
Processing: Accession = KM821764, Family = Adamaviridae
Processing: Accession = KP153447, Family = Adamaviridae
Processing: Accession = KU043397, Family = Adamaviridae
Processing: Accession = KM821755, Family = Adamaviridae
Processing: Accession = FJ959082, Family = Adamaviridae
Processing: Accession = KM573776, Family = Adamaviridae
Processing: Accession = KM573767, Family = Adamaviridae
Processing: Accession = KT732823, Family = Adamaviridae
Processing: Accession = KT732819, Family = Adamaviridae
Processing: Accession = KU043406, Family = Adamaviridae
Processing: Accession = KF738877, Family = Adamaviridae
Processing: Accession = KT149398, Family = Adamaviridae
Processing: Accession = KM598396, Family = Adamaviridae
Processing: Accession = KP153483, Family = Adamaviridae
Processing: Accession = KP153468, Family = Adama

In [29]:
search_handle = Entrez.esearch(db="nuccore", term='MW601953', retmax=1)

In [30]:
search_results = Entrez.read(search_handle)

In [31]:
nucleotide_id = search_results["IdList"][0]

In [32]:
fetch_handle = Entrez.efetch(db="nuccore", id=nucleotide_id, rettype="ft", retmode="text")
feature_table = fetch_handle.read()
fetch_handle.close()

In [36]:
feature_table.splitlines()

['>Feature gb|MW601953.1|',
 '1\t9\trep_origin',
 '67\t1188\tgene',
 '\t\t\tgene\tcap',
 '67\t1188\tCDS',
 '\t\t\tproduct\tcapsid protein',
 '\t\t\tprotein_id\tgb|QXF14475.1|',
 '2541\t1456\tgene',
 '\t\t\tgene\trep',
 '2541\t1456\tCDS',
 '\t\t\tproduct\treplication-associated protein',
 '\t\t\tprotein_id\tgb|QXF14476.1|',
 '']

In [34]:
        protein_ids = []
        for line in feature_table.splitlines():
            if "protein_id" in line:
                parts = line.split()
                if len(parts) > 1:
                    protein_id = parts[1].strip('"')
                    protein_ids.append(protein_id)

In [39]:
        for protein_id in protein_ids:
            print(f"Downloading: {protein_id}")

Downloading: gb|QXF14475.1|
Downloading: gb|QXF14476.1|


In [40]:
fetch_handle = Entrez.efetch(db="protein", id="gb|QXF14475.1|", rettype="fasta", retmode="text")

In [41]:
fasta_data = fetch_handle.read()

In [42]:
fasta_data

'>QXF14475.1 capsid protein [Porcine associated circular DNA virus-2]\nMVRRYYKRNRRRYGRSRRKVRMSKSLFKRRRYRRTRRRFRKSKGLTVSGNRESCLEESSFIPMATEVYGS\nYVGTLRDGFYQWSINPLPAGVCEMPGDSTYTGRWEAAGTDCFKRGIGKKEEALMLVDLPICMSQLSVRNL\nVRCDTLFELCKSYRIVSVSCTFTVPERTSDGPNHNLYLMWTHLPKCRAADAESCFGLVCEEVAHISGQYS\nPTVNGWNWIANPADIAEACSCDGIENGKNGWIRKQLAYNNPVTISWRPRHAKIIASHKNYTDSGVKAGQA\nTTYNKVLPDFAMDLKMSRGYLPTDIDDRALTERQFWMGPCIRLIDGDIKASEPVPETSALNIFDRYGIRV\nTTTVKVRFKGMNTSDPIFPNYQA\n\n'

In [43]:
fasta_file.write(fasta_data)

NameError: name 'fasta_file' is not defined

In [11]:
# Iterate over each row
for _, row in df.iterrows():
    project, family = row[0], row[1]  # Read Accession and Family

    print(f"Processing: Accession = {project}, Family = {family}")

    # Create a directory for the family
    family_dir = os.path.join(dir_to_download, family)
    os.makedirs(family_dir, exist_ok=True)

    # Output files
    out_1 = os.path.join(family_dir, f"{project}.txt")  # Stores protein IDs
    out_2 = os.path.join(family_dir, f"{family}.fa")    # Stores protein sequences

    # Search for nucleotide records
    try:
        search_handle = Entrez.esearch(db="nuccore", term=project, retmax=1)
        search_results = Entrez.read(search_handle)
        search_handle.close()

        if not search_results["IdList"]:
            print(f"No results found for {project}")
            continue

        nucleotide_id = search_results["IdList"][0]
        print(f"Found nucleotide ID: {nucleotide_id}")

        # Fetch feature table (ft) to extract protein IDs
        fetch_handle = Entrez.efetch(db="nuccore", id=nucleotide_id, rettype="ft", retmode="text")
        feature_table = fetch_handle.read()
        fetch_handle.close()

        # Extract protein IDs from the feature table
        protein_ids = []
        for line in feature_table.splitlines():
            if "protein_id" in line:
                parts = line.split()
                if len(parts) > 1:
                    protein_id = parts[1].strip('"')
                    protein_ids.append(protein_id)

        if not protein_ids:
            print(f"No protein IDs found for {project}")
            continue

        # Retrieve and save protein sequences
        with open(out_2, "a") as fasta_file:
            for protein_id in protein_ids:
                print(f"Downloading: {protein_id}")
                try:
                    fetch_handle = Entrez.efetch(db="protein", id=protein_id, rettype="fasta", retmode="text")
                    fasta_data = fetch_handle.read()
                    fetch_handle.close()
                    fasta_file.write(fasta_data)
                except Exception as e:
                    print(f"Error fetching protein {protein_id}: {e}")

    except Exception as e:
        print(f"Error processing {project}: {e}")
    

<generator object DataFrame.iterrows at 0x2addd8a52df0>